<a href="https://colab.research.google.com/github/VyacheslavGusev/Rec_sys/blob/main/HW_Practice_Rec_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [ ]:
movies = pd.read_csv("movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings =pd.read_csv("ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
mean_rating_user = ratings.groupby('userId').agg({'rating':['mean']}).reset_index()
mean_rating_film = ratings.groupby('movieId').agg({'rating':['mean']}).reset_index()
mean_rating_film = mean_rating_film.rename(columns = {'rating': 'rating_f'})

In [ ]:
mean_rating_user.columns = mean_rating_user.columns.map(''.join)

In [ ]:
mean_rating_film.columns = mean_rating_film.columns.map(''.join)

In [ ]:
mean_rating_user.head()

,userId,ratingmean
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364


In [ ]:
mean_rating_film.head()

,movieId,rating_fmean
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [ ]:
joined_ratings = ratings.join(mean_rating_user.set_index('userId'), on='userId')


In [ ]:
joined_ratings = joined_ratings.join(mean_rating_film.set_index('movieId'), on='movieId')

In [ ]:
joined_ratings.head()

,userId,movieId,rating,timestamp,ratingmean,rating_fmean
0,1,1,4.0,964982703,4.366379,3.920930
1,1,3,4.0,964981247,4.366379,3.259615
2,1,6,4.0,964982224,4.366379,3.946078
3,1,47,5.0,964983815,4.366379,3.975369
4,1,50,5.0,964982931,4.366379,4.237745


In [ ]:
tags = pd.read_csv("tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [ ]:
tfidf = TfidfVectorizer()
geners_tfidf = tfidf.fit_transform(movie_genres)
geners_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [ ]:
gen_tfidf = pd.DataFrame(geners_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [ ]:
movies_gen = movies.join(gen_tfidf)

In [ ]:
movies_gen.head()

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
movies_with_tags = movies.merge(tags, on='movieId')
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932


In [ ]:
movies_with_tags.dropna(inplace=True)

In [ ]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').replace('=', '').lower()

tag_strings = []
movies = []

for movie, group in tqdm(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
    movies.append(movie)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [ ]:
tag_strings[:10]

['artistic funny humorous inspiring intelligent quirky romance zooeydeschanel',
 'lawyers',
 'creepy suspense',
 'shakespearesortof',
 'dogs remake',
 'disney',
 'terrorism',
 'court claustrophobic confrontational earnest gooddialogue greatscreenplay gritty motivational thoughtprovoking',
 'stranded',
 'markruffalo']

In [ ]:
tfidf_tag = TfidfVectorizer()
tag_tfidf = tfidf_tag.fit_transform(tag_strings)
tag_tfidf

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [ ]:
tags_tfidf = pd.DataFrame(tag_tfidf.toarray(), columns=tfidf_tag.get_feature_names_out())
tags_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Columns: 1472 entries, 06oscarnominatedbestmovieanimation to zooeydeschanel
dtypes: float64(1472)
memory usage: 17.7 MB


In [ ]:
movies_tags = movies_with_tags.join(tags_tfidf)
movies_tags.head()

,movieId,title,genres,userId,tag,timestamp,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420342
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
movies_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3683 entries, 0 to 3682
Columns: 1478 entries, movieId to zooeydeschanel
dtypes: float64(1472), int64(3), object(3)
memory usage: 41.7+ MB


In [ ]:
# соединим таблицу с рейтингами и названиями фильмов
joined_ratings = joined_ratings.join(movies_gen.set_index('movieId'), on='movieId')

In [ ]:
joined_ratings = pd.merge(joined_ratings, movies_tags, left_on=['userId', 'movieId'], right_on=['userId', 'movieId'])

In [ ]:
joined_ratings = joined_ratings.dropna()

In [ ]:
joined_ratings.head()

,userId,movieId,rating,timestamp_x,ratingmean,rating_fmean,title_x,genres_x,action_x,adventure_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
10,18,431,4.0,1462138790,3.732072,3.738095,Carlito's Way (1993),Crime|Drama,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,18,431,4.0,1462138790,3.732072,3.738095,Carlito's Way (1993),Crime|Drama,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,18,431,4.0,1462138790,3.732072,3.738095,Carlito's Way (1993),Crime|Drama,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,18,1221,5.0,1460242083,3.732072,4.259690,"Godfather: Part II, The (1974)",Crime|Drama,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,18,1221,5.0,1460242083,3.732072,4.259690,"Godfather: Part II, The (1974)",Crime|Drama,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = joined_ratings.drop(['title_x', 'genres_x', 'title_y',  'genres_y', 'tag', 'rating', 'timestamp_x'], axis= 1 , inplace= False)

In [ ]:
X.columns = X.columns.str.replace(' ','')

In [ ]:
X = X.fillna(0)

In [ ]:
X.head()

,userId,movieId,ratingmean,rating_fmean,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
10,18,431,3.732072,3.738095,0.0,0.0,0.0,0.0,0.0,0.863943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,18,431,3.732072,3.738095,0.0,0.0,0.0,0.0,0.0,0.863943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,18,431,3.732072,3.738095,0.0,0.0,0.0,0.0,0.0,0.863943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,18,1221,3.732072,4.259690,0.0,0.0,0.0,0.0,0.0,0.863943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,18,1221,3.732072,4.259690,0.0,0.0,0.0,0.0,0.0,0.863943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479 entries, 10 to 3474
Columns: 1497 entries, userId to zooeydeschanel
dtypes: float64(1494), int64(3)
memory usage: 16.9 MB


In [ ]:
y =pd.Series(data=joined_ratings['rating'])
y

10      4.0
11      4.0
12      4.0
13      5.0
14      5.0
       ... 
3466    5.0
3467    4.0
3468    3.5
3473    5.0
3474    5.0
Name: rating, Length: 1479, dtype: float64

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = make_pipeline(
    MinMaxScaler(feature_range=(0, 1)), Lasso(alpha=.01))

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('lasso', Lasso(alpha=0.01))])

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train,y_pred_train))
print('RMSE:', rmse_train)

RMSE: 0.7132788968440799


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print('RMSE:', rmse)

RMSE: 0.6930029355932622


In [ ]:
from sklearn import tree

In [ ]:
model = make_pipeline(
    MinMaxScaler(feature_range=(0, 1)),tree.DecisionTreeRegressor(splitter='best', max_depth=12))

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('decisiontreeregressor', DecisionTreeRegressor(max_depth=12))])

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train,y_pred_train))
print('RMSE:', rmse_train)

RMSE: 0.12190111052108493


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print('RMSE:', rmse)

RMSE: 0.33462697809607356


Вывод: дерево решений справилось с решением лучше, чем линейная модель. Можно улучшить метрики, подбирая гиперпараметры модели

## Коллаборативная фильтрация

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162675 sha256=c0ae9974a06e634353f4834f1ae35f87fbb17165591e5e315a8609204970f020
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise.model_selection import cross_validate
from surprise import Dataset, SVD, SVDpp
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [ ]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [ ]:
proc_df = movies.merge(ratings, on='movieId').reset_index(drop=True)
proc_df.dropna(inplace=True)
proc_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': proc_df.userId,
    'iid': proc_df.title,
    'rating': proc_df.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
ratings.rating.min()

0.5

In [ ]:
ratings.rating.max()

5.0

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

#### ModelSVD

In [ ]:
algo =SVD(n_epochs=30,  lr_all=0.01, reg_all=0.04)

In [ ]:
%%time
cross_validate(algo, data, measures=["RMSE"], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8697  0.8552  0.8691  0.8612  0.8537  0.8618  0.0067  
Fit time          2.39    2.31    3.07    2.29    2.32    2.48    0.30    
Test time         0.12    0.11    0.32    0.11    0.30    0.19    0.10    
CPU times: user 14 s, sys: 58.8 ms, total: 14 s
Wall time: 14.1 s


{'test_rmse': array([0.86971696, 0.85517409, 0.86907293, 0.86115932, 0.85372808]),
 'fit_time': (2.3880083560943604,
  2.314530849456787,
  3.073482036590576,
  2.286851167678833,
  2.3204894065856934),
 'test_time': (0.11551737785339355,
  0.11349678039550781,
  0.32294702529907227,
  0.11365532875061035,
  0.29903268814086914)}

#### Model SVD++

In [ ]:
algo = SVDpp(cache_ratings=True)

In [ ]:
%%time
cross_validate(algo, data, measures=["RMSE"], cv=5, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8547  0.8680  0.8635  0.8550  0.8620  0.8606  0.0051  
Fit time          69.51   70.61   70.46   70.31   70.10   70.20   0.38    
Test time         11.92   11.73   11.89   11.86   11.91   11.86   0.07    
CPU times: user 6min 47s, sys: 1.47 s, total: 6min 49s
Wall time: 6min 51s


{'test_rmse': array([0.85473866, 0.8679696 , 0.8634853 , 0.85499444, 0.86198824]),
 'fit_time': (69.51225900650024,
  70.61056351661682,
  70.45957255363464,
  70.31001567840576,
  70.10189914703369),
 'test_time': (11.916448831558228,
  11.730356931686401,
  11.891523122787476,
  11.860767126083374,
  11.907343864440918)}

Вывод: на двух моделях получили требуемый уровень RMSE. Модель SDV работает быстрее в 30 раз